In [24]:
import torch
cuda_avail = torch.cuda.is_available()
device = torch.device("cuda" if cuda_avail else "cpu")
cerealsModel = torch.load("./trained/tasks/cereals_epoch20_2020-04-07.pkl", map_location=torch.device(device))
coffeeModel = torch.load("./trained/tasks/coffee_epoch30_2020-04-07.pkl", map_location=torch.device(device))
friedeggModel = torch.load("./trained/tasks/friedegg_epoch30_2020-04-07.pkl", map_location=torch.device(device))
juiceModel = torch.load("./trained/tasks/juice_epoch40_2020-04-07.pkl", map_location=torch.device(device))
milkModel = torch.load("./trained/tasks/milk_epoch40_2020-04-07.pkl", map_location=torch.device(device))
pancakeModel = torch.load("./trained/tasks/pancake_epoch40_2020-04-07.pkl", map_location=torch.device(device))
salatModel = torch.load("./trained/tasks/salat_epoch30_2020-04-07.pkl", map_location=torch.device(device))
sandwichModel = torch.load("./trained/tasks/sandwich_epoch10_2020-04-07.pkl", map_location=torch.device(device))
scrambledeggModel = torch.load("./trained/tasks/scrambledegg_epoch40_2020-04-07.pkl", map_location=torch.device(device))
teaModel = torch.load("./trained/tasks/tea_epoch10_2020-04-07.pkl", map_location=torch.device(device))

modelDictionary = {
    "cereals" : cerealsModel,
    "coffee" : coffeeModel,
    "friedegg" : friedeggModel,
    "juice" : juiceModel,
    "milk" : milkModel,
    "pancake" : pancakeModel,
    "salat" : salatModel,
    "sandwich" : sandwichModel,
    "scrambledegg" : scrambledeggModel,
    "tea" : teaModel
}

In [5]:
import pickle
def loadDict(filename):
    pkl_file = open(filename, 'rb')
    dict = pickle.load(pkl_file)
    pkl_file.close()
    return dict

cerealsMapping = loadDict("./trained/tasks/dictionary/cereals.pkl")
coffeeMapping = loadDict("./trained/tasks/dictionary/coffee.pkl")
friedeggMapping = loadDict("./trained/tasks/dictionary/friedegg.pkl")
juiceMapping = loadDict("./trained/tasks/dictionary/juice.pkl")
milkMapping = loadDict("./trained/tasks/dictionary/milk.pkl")
pancakeMapping = loadDict("./trained/tasks/dictionary/pancake.pkl")
salatMapping = loadDict("./trained/tasks/dictionary/salat.pkl")
sandwichMapping = loadDict("./trained/tasks/dictionary/sandwich.pkl")
scrambledeggMapping = loadDict("./trained/tasks/dictionary/scrambledegg.pkl")
teaMapping = loadDict("./trained/tasks/dictionary/tea.pkl")

mappingDictionary = {
    "cereals" : cerealsMapping,
    "coffee" : coffeeMapping,
    "friedegg" : friedeggMapping,
    "juice" : juiceMapping,
    "milk" : milkMapping,
    "pancake" : pancakeMapping,
    "salat" : salatMapping,
    "sandwich" : sandwichMapping,
    "scrambledegg" : scrambledeggMapping,
    "tea" : teaMapping
}

In [1]:
from utils.read_datasetBreakfast import load_testdata
import os

COMP_PATH = ''

test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
DATA_folder =  os.path.join(COMP_PATH, 'Data/') #Frame I3D features for all videos
data_feat = load_testdata(test_split, DATA_folder)

In [8]:
segment_idx = []
with open("./utils/test_segment.txt") as f:
    while True:
        line = f.readline()
        if not line:
            break
        segment_idx.append(line[:-1].split(" "))

In [18]:
data_feat_seg = []
for i in range(len(data_feat)):
    videoFrames = []
    for j in range(len(segment_idx[i])-1):
        videoFrames.append(data_feat[i][1][int(segment_idx[i][j]):int(segment_idx[i][j+1])])
    data_feat_seg.append((data_feat[i][0], videoFrames))

In [42]:
import numpy as np
from Dataset.SegmentDataset import SegmentDataset
import torch.utils.data as tud

batch_size = 1
outputs = []
for i in data_feat_seg:
    task = i[0]
    fake_labels = np.ones(len(i[1]))
    test_dataset = SegmentDataset(i[1], fake_labels, seed = 2)
    test_dataloader = tud.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    labelMapping = mappingDictionary[task]
    label_nums = len(labelMapping)
    model = modelDictionary[task].to(device)
    model.eval()
    for x, labels in test_dataloader:
        x = x.to(device)
        with torch.no_grad():
            output = model(x).to(device)
            output = output.view(-1, label_nums)
            predict_label = torch.max(output, 1)[1]
            original_label = labelMapping[predict_label.item()]
            #print(task +" : "+ str(original_label))
            outputs.append(original_label)

cereals : 4
cereals : 2
cereals : 2
cereals : 2
coffee : 6
coffee : 5
coffee : 6
friedegg : 12
friedegg : 12
friedegg : 13
friedegg : 15
milk : 24
milk : 24
milk : 24
milk : 25
salat : 14
salat : 32
salat : 14
salat : 32
salat : 33
salat : 32
salat : 34
salat : 32
salat : 33
salat : 35
sandwich : 18
sandwich : 36
sandwich : 37
sandwich : 37
sandwich : 41
sandwich : 37
sandwich : 37
sandwich : 39
tea : 46
tea : 46
tea : 46
cereals : 4
cereals : 2
cereals : 2
cereals : 3
coffee : 6
coffee : 5
coffee : 6
milk : 6
milk : 24
milk : 24
milk : 25
sandwich : 39
sandwich : 36
sandwich : 37
sandwich : 37
sandwich : 18
sandwich : 39
sandwich : 36
sandwich : 39
tea : 45
tea : 46
tea : 46
cereals : 4
cereals : 2
cereals : 2
cereals : 3
coffee : 6
coffee : 5
coffee : 5
friedegg : 12
friedegg : 17
friedegg : 11
friedegg : 15
milk : 24
milk : 24
milk : 24
milk : 25
salat : 33
salat : 33
salat : 32
salat : 32
salat : 33
salat : 32
salat : 34
salat : 32
salat : 33
salat : 35
sandwich : 36
sandwich : 36


In [41]:
print(len(outputs))
            

1284


In [43]:
path = './results/tasks/test_result_bytask.csv'
f = open(path, 'w+')
f.write('Id,Category\n')

counter = 0
for i in outputs:
        f.write(f'{counter},{i}\n')
        counter += 1
f.close()

